In [12]:
# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import datetime, os
import time

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, transforms, build_detection_test_loader
from detectron2.engine import DefaultTrainer

In [2]:
import numpy as np
from detectron2.structures import BoxMode

def get_cracks_dicts(img_dir):
    dataset_dicts = []
    image_paths = list(sorted(os.listdir(img_dir + 'image/')))
    mask_paths = None
    if os.path.exists(img_dir+'mask/'):
        mask_paths = list(sorted(os.listdir(img_dir+'mask/')))
    for idx, v in enumerate(image_paths):
        record = {}
        
        filename = os.path.join(img_dir, 'image/', v)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        if mask_paths:
            mask_path = img_dir + "mask/" + mask_paths[idx]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # HxWxC
            contours,_=cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

            objs = []
            for cnt in contours : 
                approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True) 
                if approx.shape[0] < 3:
                    continue
                # cv2.drawContours(image, [approx], 0, (0, 0, 255), 1) 
                # get bounding box coordinates for each mask
                px = [pos[0][0] for pos in approx]
                py = [pos[0][1] for pos in approx]
                poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                poly = [p for x in poly for p in x]
                obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
            record["sem_seg_file_name"] = mask_path # for segmentation evaluator
        else:
            record["annotations"] = [{
                    "bbox": [0, 0, 0, 0],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [(0, 0), (0, 0), (0, 0)],
                    "category_id": 0,
                    "iscrowd": 0
                }]
            record["sem_seg_file_name"] = '' # for segmentation evaluator
        dataset_dicts.append(record)
    return dataset_dicts

In [7]:
# test_dir = "/home/aicenter/Documents/hsu/data/CrackForest-dataset/test/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/result_256x256/crack/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/result_256x256/noncrack/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/256x256/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/256x256_Trans/"
test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets_Trans/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_seg/"
# test_dir = "/home/aicenter/Documents/hsu/data/test_dataset/test_seg_Trans/"

In [8]:
DatasetCatalog.register("cracks_test", lambda d= "test": get_cracks_dicts(test_dir))
MetadataCatalog.get("cracks_test").set(thing_classes=["crack"], thing_colors=[(96, 151, 255)])

namespace(name='cracks_test',
          thing_classes=['crack'],
          thing_colors=[(96, 151, 255)])

In [13]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (crack)
cfg.MODEL.WEIGHTS = "./models/mask_101_concrete_50000_0526.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.DATASETS.TEST = ("cracks_test", )
predictor = DefaultPredictor(cfg)

In [47]:
# DRAW ONLY SEGEMENTATION
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

mask_color = np.array((0, 255, 0), dtype="uint8")
for idx, d in enumerate(dataset_dicts):
    file_name = d["file_name"]
    im = cv2.imread(file_name)
    if idx%20==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    roi = im[mask]
    im[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
    # save_dir = './result/segmentation/'
    save_dir = test_dir + '/result/'
    cv2.imwrite(save_dir + file_name.split('/')[-1], im)

/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_0.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_1.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_10.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_100.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_101.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_102.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_103.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_104.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_11.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_2.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_20.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/cr

In [21]:
# DRAW ALL
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])

save_dir = test_dir + '/result/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
for d in dataset_dicts:
    file_name = d["file_name"]
    im = cv2.imread(file_name)
    print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
        
    v = Visualizer(im[:, :, ::-1],
                   metadata=crack_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # save_dir = './result/segmentation/'
    cv2.imwrite(save_dir + file_name.split('/')[-1], v.get_image()[:, :, ::-1])

/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_0.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_1.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_10.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_100.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_101.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_102.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_103.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_104.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_11.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_2.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/crop_20.png
/home/aicenter/Documents/hsu/data/test_dataset/tw_labels/datasets/image/cr

In [65]:
crop_w, crop_h = 256, 256
ceci_dir = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/%sx%s/image/' %(crop_w, crop_h)
source_path = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/gt_640.png'

height, width, _ = cv2.imread(source_path).shape
x_num, y_num = int(width/crop_w), int(height/crop_h)

mask_color = np.array((0, 0, 255), dtype="uint8") 
whole_image = np.array([])
v_im = np.array([])
for filename in sorted(os.listdir(ceci_dir)):
    count_id = int(filename.split('.')[0])
    image_path = ceci_dir+filename
    im = cv2.imread(image_path)
    outputs = predictor(im)
    
    masks = outputs['instances'].pred_masks.cpu().numpy()
    # masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    masks = masks[outputs['instances'].scores.cpu().numpy() > 0.5]
    mask = np.sum(masks, axis=0) > 0
    roi = im[mask]
    im[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
    if (count_id+1)%y_num == 0:
        whole_image = v_im if whole_image.size == 0 else np.hstack((whole_image, v_im))
        v_im = np.array([])
    else:
        v_im = im if v_im.size == 0 else np.vstack((v_im, im))

save_path = source_path.split('.png')[0] + '_seg.png'
print('Saved at', save_path)
cv2.imwrite(save_path, whole_image)

Saved at /home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/gt_640_seg.png


True

In [9]:
from detectron2.evaluation.sem_seg_evaluation import SemSegEvaluator
from detectron2.evaluation import inference_on_dataset

In [28]:
val_dir = '/home/aicenter/Documents/hsu/data/validation/'
DatasetCatalog.clear()
DatasetCatalog.register("cracks_val", lambda d= "val": get_cracks_dicts(val_dir))

In [55]:
val_cfg = get_cfg()
val_cfg.DATALOADER.NUM_WORKERS = 2
val_cfg.MODEL.WEIGHTS = "./models/mask_101_concrete_50000_0526.pth"
val_cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
val_cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
val_cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (crack)
val_cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.DATASETS.TRAIN = ("crack_train",)
val_cfg.DATASETS.TEST = ("cracks_val", )

In [56]:
valer = DefaultTrainer(val_cfg) 
valer.resume_or_load(resume=False)
evaluator = SemSegEvaluator(dataset_name='cracks_val', distributed=False, num_classes=1, output_dir='./output_val/')
val_loader = build_detection_test_loader(val_cfg, "cracks_val")
inference_on_dataset(valer.model, val_loader, evaluator)

[05/27 17:51:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

AssertionError: 